# Coding Theory for Storage and Networks
### Lab on VT Codes
Student: David de Andrés Hernández

## Task 1: VT Decoding

Varshamov-Tenengolts (VT) codes have been shown to correct a single deletion or insertion error. In this task, we develop a binary VT decoder that corrects a single deletion or insertion error. We use a code of length n = 15 and checksum a = 0, i.e.
$$
\mathcal{VT}_0(n) = \{ c = (c_0, c_1, \cdots , c_{n−1}) : \sum_{i=0}^{n-1}(i+1)c_i \equiv 0 \text{ mod } (n+1) \}
$$

In [3]:
n = 15
a = 0

1. Consider the codeword $c = (0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0)$. Check, if it is a valid 
codeword by computing its checksum

In [4]:
c = vector(GF(2),[0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0])
S = [mod(((idx + 1)* value ),(n+1)) for idx, value in enumerate(vector(ZZ, c))]
S = mod(sum(S), n+1)

print('The codeword c is %s.' % c)
if S==0:
    print('c is a valid codeword.\n')



The codeword c is (0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0).
c is a valid codeword.



2. Implement a VT decoder(r, n) that can correct a single deletion or insertion.

In [376]:
# Implement a VT_decoder(r,n) that can correct a single deletion or insertion.
def VT_Decoder(r, n):
    """
    Binary Varshamov-Tenengolts decoder for a single deletion
    Inputs:
        r: received word
        n: length of the code
    Ouput: 
        c_hat: extimated codeword
        pos: where deletion or insertion happens
        err_type: "deletion" or "insertion"
    """
    # ========
    # Block of your implementation
    S = sum([mod(((idx + 1)* value ),(n+1)) for idx, value in enumerate(vector(ZZ, r))])
    print(f'The checksum of r is {S}')
    w = r.hamming_weight()
    print(f'The weight of r is {w}')
    p = -1
    if len(r) < n: # decode single deletion
        err_type = 'deletion'
        S = -S
        print(f'The deficiency S: {S} and weight w: {w}')
        if int(S) <= int(w): # s=0
            # print('s = 0')
            s = 0
            R1 = S
            ones=0
            for i in range(len(r)-1, -1, -1):
                # print(f'counted 1s: {ones}')
                # print(f'r[{i}]: {r[i]}')
                # print(f'weight to the right: {r[i:].hamming_weight()}')
                if r[i:].hamming_weight() == R1:
                    p = i
                    break
            r = r.list()
            r.insert(p, s)
            c_hat = vector(GF(2), r)
        else: # s=1
            s = 1
            L0 = S - w - 1
            for i in range(r):
                if(list(r[:i]).count(0)== L0):
                    p = i
                    break
            r = list(r)
            r.insert(p, s)
    elif len(r) > n: # decode single insertion
        err_type = 'insertion'
        print(f'The surplus S: {S} and weight w: {w}')
        if int(S) == int(w):
            r = list(r)
            r.pop(0)
            p = 0
        elif int(S) < int(w):
            print('s = 0')
            R1 = S
            for i in range(len(r)-1, -1, -1):
                if r[i:].hamming_weight() == R1:
                    p = i
                    break
            for j in range(int(p), -1, -1):
                print(f'j: {j}')
                if ( r[j] == 0):
                    p = j
                    r = list(r)
                    r.pop(p)
                    break
        else:
            print('s = 1')
            s = 1
            L0 = S - w
            for i in range(len(r)):
                if(list(r[:i]).count(0)== L0):
                    p = i
                    break
            for j in range(int(p), len(r)):
                if ( r[j] == 1):
                    p = j
                    r = list(r)
                    r.pop(p)
                    break
    c_hat = vector(GF(2), r)
    print(f'position p = {p}')
    print(f'c_hat = {c_hat}')
    
    return c_hat, p, err_type


3. (a) A random deletion occurs to the codeword c resulting in $r_{del} \in \mathbb{F}_{n−1}^2$. Correct the error using your VT decoder().
(b) A random insertion occurs to the codeword c resulting in $r_{ins} \in \mathbb{F}_{n−1}^2$. Correct the error using your VT decoder().

In [404]:
# (2) A random insertion occurs to the codeword c. Correct the error using your VT_decoder. 
# A random insertion occurs
ins_pos = randint(0, n-1)
ins_val = GF(2).random_element()
r_ins = vector(c[:ins_pos].list()+[ins_val]+c[ins_pos:].list())
print('Received word with insertion %s at position %s is \n%s.\n' % (ins_val, ins_pos, r_ins))

# Correct the error and find the postion and type of the error
c_hat, pos, err_type = VT_Decoder(r_ins, n)

assert c_hat == c, "Decoding of insertion is not correct.\n"
print('Insertion is decoded corretly.\n')

Received word with insertion 1 at position 1 is 
(0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0).

The checksum of r is 8
The weight of r is 7
The surplus S: 8 and weight w: 7
s = 1
position p = 1
c_hat = (0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0)
Insertion is decoded corretly.



In [313]:
# (1) A random deletion occurs to the codeword c. Correct the error using your VT_decoder. 
# A random deletion occurs
del_pos = randint(0, n-1)
r_del = c.list()
r_del.pop(del_pos) # del r_del[del_pos] # alternative
r_del = vector(r_del)
print('Received word with deletion at position %s \nis %s.\n' % (del_pos, r_del))

# Correct the error and find the postion and type of the error
c_hat, pos, err_type = VT_Decoder(r_del, n)

assert c_hat == c, "Decoding of deletion is not correct.\n"
print('Deletion is decoded correctly.\n')

Received word with deletion at position 9 
is (0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0).

The checksum of r is 4
The weight of r is 5
The deficiency S: 12 and weight w: 5
position p = 9
c_hat = (0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0)
Deletion is decoded correctly.

